In [1]:
import os
import json
import pickle
from datetime import datetime
from blockchain_parser.blockchain import Blockchain

def create_directory_if_not_exists(directory):
    if not os.path.exists(directory):
        os.makedirs(directory)

def save_transaction_data_to_json():
    blockchain_path = os.path.expanduser('~/.bitcoin/blocks')  # 指定比特幣 Core 區塊鏈數據的路徑
    blockchain = Blockchain(blockchain_path)

    start_date = datetime(2023, 6, 27, 0, 0)
    end_date = datetime(2023, 6, 27, 1, 1)

    transaction_count = 0
    directory_count = 0
    index_data = {}
    coinbase_data = {}

    address_directory = '../address'
    create_directory_if_not_exists(address_directory)

    for block in blockchain.get_unordered_blocks():
        # Check if the block timestamp is within the specified date range
        if start_date <= block.header.timestamp <= end_date:
            for tx in block.transactions:
                transaction_hash = tx.hash
                directory = str(directory_count).zfill(4)

                # Create address directory if it doesn't exist
                create_directory_if_not_exists(directory)

                output = tx.outputs[0]
                if output.addresses:
                    # 因為不能是序列化 list 所以先用 tx.inputs[0] 的交易哈希值當地址
                    # 未來這個要改成真的 address
                    
                    # Address 類似 Address(addr=1K6KoYC69NnafWJ7YgtrpwJxBLiijWqwa6)
                    # 將 output_address 轉換為字串
                    output_str = str(output.addresses[0])

                    # 使用字串處理方法提取子字串
                    start_index = output_str.index("=") + 1
                    end_index = output_str.index(")", start_index)
                    address = output_str[start_index:end_index]

                    print(address)

                    address_directory_path = os.path.join(address_directory, f'{tx.inputs[0]}.json')
                    if not os.path.exists(address_directory_path):
                        with open(address_directory_path, 'w') as address_file:
                            json.dump([address], address_file, indent=4)

                    file_path = os.path.join(directory, f'{address}.json')

                with open(file_path, 'w') as file:
                    json.dump({
                        'timestamp': block.header.timestamp.strftime('%Y-%m-%d %H:%M:%S'),
                        'hash': tx.hash,
                        'inputs': [str(input_) for input_ in tx.inputs],
                        'outputs': [str(output) for output in tx.outputs],
                        'coinbase': tx.inputs[0]
                    }, file, indent=4, default=str)

                # Update index data
                index_data[transaction_hash] = directory

                # Update coinbase data
                coinbase_data[tx.inputs[0]] = directory

                transaction_count += 1
                if transaction_count % 10000 == 0:
                    directory_count += 1

    # Append the "index" field to the index_data
    index_data = {"index": index_data}

    # Append the "coinbase" field to the coinbase_data
    coinbase_data = {"tx2blk": coinbase_data}

    # Save index data to index.json
    with open('index.json', 'w') as index_file:
        json.dump(index_data, index_file, indent=4)

    # Save coinbase data to coinbase.pkl
    with open('../coinbase.pkl', 'wb') as coinbase_file:
        pickle.dump(coinbase_data, coinbase_file)

save_transaction_data_to_json()


In [1]:
import os
import bitcoin
from datetime import datetime
from blockchain_parser.blockchain import Blockchain

# 创建Blockchain对象
blockchain_path = os.path.expanduser('~/.bitcoin/blocks')  # 指定比特币 Core 区块链数据的路径
blockchain = Blockchain(blockchain_path)

# Define the start and end dates
start_date = datetime(2023, 6, 27, 0, 0)
end_date = datetime(2023, 6, 27, 1, 1)

# Iterate over the blocks in the blockchain
addresses = []
for block in blockchain.get_unordered_blocks():
    # Check if the block timestamp is within the specified range
    if start_date <= block.header.timestamp <= end_date:
        # Iterate over the transactions in the block
        for tx in block.transactions:
            if tx.outputs:
                output = tx.outputs[0]
                address = output.addresses[0].address
                addresses.append(address)
                print("Address:", address)
                break

AttributeError: module 'bitcoin' has no attribute 'params'